In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob



In [2]:
# Define our useful functions for the pipeline
# Function to calibrate our camera. Returns the camera parameter matrix and the distortion matrix
# Takes in the directory of the calibration images and the corner dimenstions of the chessboard
def calibrateCam(data_dir="camera_cal/calibration*.jpg", nx=9, ny=6):
    # Get all the calibration images using the glob library
    images = glob.glob(data_dir)
    
    print("Calibrating from images in " + data_dir)
    
    obj_points = [] # 3D points in real world space
    img_points = [] # 2D points in the image plane
    
    # Prepare standard object points to append. These should be the same for every image
    objp = np.zeros((nx*ny, 3), np.float32)
    # Reshape into x,y coordinates
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
    
    # Loop through all the images to obtain the object points and image points
    for fname in images:
        # Read in the image
        img = mpimg.imread(fname)
        
        # Convert to grayscale (Use RGB because its read in by matplotlib)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # Find the chessboard corners of the image
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
        
        # If corners were found
        if ret == True:
            # Append corners and objpoints
            img_points.append(corners)
            obj_points.append(objp)
            
            
            # Draw the corners just to check
            # img = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            # plt.imshow(img)
            
    # Use the found object points and image points to calibrate the camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)
    
    return mtx, dist

# Uses color thresholding and gradients to find the strongest lines in the image
# Takes in the image, the threhsold for the sobel filters and the threhsold for the color
# For color we'll be using the s channel of the the HLS color space
def getBinaryLanes(img, s_thresh=(170, 255), sob_thresh=(20, 100)):
    img = np.copy(img)
    
    # Convert to the HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    # Grab just the l and s channels
    h_chan = hls[:,:,0]
    l_chan = hls[:,:,1]
    s_chan = hls[:,:,2]

    # Find the gradient in the x direction by applying a sobwl filter
    sobelx = cv2.Sobel(l_chan, cv2.CV_64F, 1, 0, ksize = 3)
    # Take the absolute value to accentuate vertical and nearly vertical lines
    abs_sobelx = np.absolute(sobelx)
    # Scale the values so they'll fit in the proper 0-255 range
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold the gradient image
    # Create a blank image the same size as the input image
    sobx_binary = np.zeros_like(scaled_sobelx)
    # Activate the pixels in the blank image based on the pixels in the scaled sobel image that fall within the threshold
    sobx_binary[(scaled_sobelx >= sob_thresh[0]) & (scaled_sobelx <= sob_thresh[1])] = 1
    
    # Threshold the s color channel
    # Create a blank image the same size as the input image
    s_chan_bin = np.zeros_like(s_chan)
    # Activate the pixels in the blank image based on the pixels in the s channel image that fall within the threshold
    s_chan_bin[(s_chan >= s_thresh[0]) & (s_chan <= s_thresh[1])] = 1
    
    # Combine the binary thresholds into one activation image
    # Activations should be from both images and don't need to match up
    combined_binary = np.zeros_like(sobx_binary)
    combined_binary[(sobx_binary == 1) | (s_chan_bin == 1)] =1
    
    return combined_binary

# Returns the birds eye view of the passed in ROI. This should isolate just the lanes and make them easy to detect later
# The ROI will serve as our src area to get transformed
def getTransformedLaneView(img, ROI):
    width = img.shape[1]
    height = img.shape[0]
    
    # Destination points to calculate the transform from the source points to
    # Top left, top right, bottom right, bottom left
    # IMPORTANT: I didn't use an offset becaue I want the image to be exclusively the lane lines. When using an offset it occasionally
    # Included the warning line and other lanes which were aggresively changing the predicted polynomials
    dst = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    
    # Get the perspective transform
    trans = cv2.getPerspectiveTransform(ROI, dst)
    
    # Inverse transform so the image can be unwarped
    trans_inv = cv2.getPerspectiveTransform(dst, ROI)
    
    # Apply the transform to the image
    warped = cv2.warpPerspective(img, trans, (width, height), flags=cv2.INTER_LINEAR)
    
    return warped, trans_inv

# Takes in the transformed binary image and identify which pixels belong to the left lane and which pixels belong to the right lane
# Returns the sets of pixels in adition to the input image with the search area displayed over it
# Utilizes sliding window technique to find the find the average location of the line in each window
def findInitialLanePixels(img):
    # Take a histogram of the full image (Might be able to cut it down to half the image like in the tutorial)
    hist = np.sum(img[img.shape[0]//2:,:], axis = 0)
    
    # Create output image to draw results on
    out_img = np.dstack((img, img, img))
    
    # Split the image in half using a midpoint
    midpoint = np.int(hist.shape[0]//2)
    
    # Starting point for the left lane will be the index of the peak in the left half of the histogram
    leftx_base = np.argmax(hist[:midpoint])
    
    # Starting point for the right lane will be the index of the peak in the right half of the histogram + the midpoint because the 
    # the returned index will be 0 < rightx_base < midpoint to get it's location on the right side of the image we just add the midpoint
    rightx_base = np.argmax(hist[midpoint:]) + midpoint
    
    # Hyperparameters for the sliding window
    # Number of windows for each lane
    num_win = 9
    # left and right search margin
    margin = 100
    # Minimum required pixels to declare a new center for the window
    min_pix = 50
    
    # Pixel height of the windows
    window_height = np.int(img.shape[0]//num_win)
    
    # Identify the x and y positions of all nonzero pixels in the image
    # The nonzero function returns a tuple of two arrays. The first array contains all the non zero y indices and the second array contains
    # all the non zero x indices
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the current position to the starting positions. Gets updated in loop below
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Empty lists to hold the good pixel indices for each lane
    left_lane_inds = []
    right_lane_inds= []
    
    # Loop through each window
    for window in range(num_win):
        # Get the top and botton y boundaries for the rectangular window
        win_y_low = img.shape[0] - window_height*(window+1) # Lower y values corresponds to the top of the window!!
        win_y_high = img.shape[0] - window_height*window # High y value corresponds to the bottom of the window
        
        # Get the low and high x values for the left and right lanes
        left_win_low_x = leftx_current - margin
        left_win_high_x = leftx_current + margin
        right_win_low_x = rightx_current - margin
        right_win_high_x = rightx_current + margin
        
        # Draw the current windows on the output image (Takes in the top left and bottom right values)
        cv2.rectangle(out_img, (left_win_low_x, win_y_low), (left_win_high_x, win_y_high), (0,255,0), 2)
        cv2.rectangle(out_img, (right_win_low_x, win_y_low), (right_win_high_x, win_y_high), (0,255,0), 2)
        
        # Identify all non zero and y pixels within the window for both lanes
        good_left_inds = ((nonzerox >= left_win_low_x) & (nonzerox < left_win_high_x) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox >= right_win_low_x) & (nonzerox < right_win_high_x) & (nonzeroy >= win_y_low) & (nonzeroy < win_y_high)).nonzero()[0]
        
        # Append the good indices
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # Update the current window x position if the good indices meet the threshold
        if len(good_left_inds > min_pix):
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds > min_pix):
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    
            
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass 
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, out_img

def searchAroundPoly(binary_warped, left_lane, right_lane):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100
    
    # Get the existing best fits from each lane
    left_fit = left_lane.current_fit
    right_fit = right_lane.current_fit

    # Grab activated pixels
    # The nonzero functions returns a tuple of two arrays. The first arrray contains the 
    # x half of nonzero x,y pixel coordinates and the second array contains the y halves
    # So to get the location of a non zero pixel you would do pix = (nonzero[0][idx], nonzero[1][idx])
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### within the +/- margin of our polynomial function ###
    # Finds all the nonzero pixels between function line +/- the margin. The margin just shifts the function left or right
    # so it's essentially parallel to the original line
    left_lane_inds = ((nonzerox > (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, left_lane, right_lane)
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
#     plt.plot(left_fitx, ploty, color='yellow')
#     plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return leftx, lefty, left_fitx, rightx, righty, right_fitx, ploty, result

# 
def fit_poly(img_shape, left_lane, right_lane):
    # Get the existing x and y pixels for each line
    leftx = left_lane.allx
    lefty = left_lane.ally
    
    rightx = right_lane.allx
    righty = right_lane.ally
    
    # Fit a second order polynomial to each with np.polyfit() 
    left_fit = np.polyfit(lefty, leftx, deg=2)
    right_fit = np.polyfit(righty, rightx, deg=2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

# Fits a second degree polynomial to the identified lane pixels for each lane
def fitPolynomial(binary_warped, left_lane, right_lane):
    # Check if good lines already exist
    if True:#left_lane.detected == False or right_lane.detected == False:
        # If they don't then find our initial lane pixels using a sliding window technique
        leftx, lefty, rightx, righty, out_img = findInitialLanePixels(binary_warped)
        
        # Fit a second order polynomial to each set of lane points
        left_fit = np.polyfit(lefty, leftx, deg=2)
        right_fit = np.polyfit(righty, rightx, deg=2)

        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty

        ## Visualization ##
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Removed because the lines were showing up in the final image and weren't needed
        # Kept for demo purposes
        # Plots the left and right polynomials on the lane lines
#         plt.plot(left_fitx, ploty, color='yellow')
#         plt.plot(right_fitx, ploty, color='yellow')

    # If boths lines already exist then we can just search around the existing polynomial
    else:
        leftx, lefty, left_fitx, rightx, righty, right_fitx, ploty, out_img = searchAroundPoly(binary_warped, left_lane, right_lane)
    
    return out_img, leftx, lefty, rightx, righty, ploty, left_fitx, right_fitx



# Takes in the points for both lines and the scaling factors and returns the radius of curvature of the line in meters
def getLineCurvatures(width, leftx, lefty, rightx, righty, x_dim, y_dim):
    # Scaling factor for x and y direction. Meters per pixel in each direction
    xm_per_pix = 3.7/x_dim
    ym_per_pix = 30/y_dim
    
    # Create new equations for each line using the new real world scale
    left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Use the max y value for where we want to measure the radius of curvature. This corresponds to where the line is closest to the car
    y_eval = 0
    
    # Implement the calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_m[0]*y_eval*ym_per_pix + left_fit_m[1])**2)**1.5)/(np.absolute(2*left_fit_m[0]))
    right_curverad = ((1 + (2*right_fit_m[0]*y_eval*ym_per_pix + right_fit_m[1])**2)**1.5)/(np.absolute(2*right_fit_m[0]))  
    
    # Get the average location of the left and right lane
    midpoint = width/2
    
    # Get the subset of line pixels that are closest to the car. In this case we get consider all pixels that are closer than 90% of the
    # y value of the closest pixels
    close_leftx = leftx[lefty > (.9 * np.max(lefty))]
    close_rightx = rightx[righty > (.9 * np.max(righty))]
    
    left_avg = np.mean(close_leftx)
    right_avg = np.mean(close_rightx)
    
    # Calculate distance from center
    lane_center = (left_avg + right_avg)/2
    offset_from_center = abs(midpoint - lane_center)*xm_per_pix
    
    return left_curverad, right_curverad, offset_from_center

In [3]:
# Main pipeline for processing each image
def pipeline(img, mtx, dist, left_lane, right_lane):
    # Undistort the image using the pre calculated camera params and distortion matrix
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    
    rgb_img = cv2.cvtColor(undist, cv2.COLOR_BGR2RGB)
    
#     plt.imsave("output_images/undistorted.jpg", undist)
    
    # Get the binary image of the detected lane lines
    combined_binary = getBinaryLanes(undist)
    
#     plt.imsave("output_images/binary_scene.jpg", combined_binary, cmap="gray")

    # Image dimensions for easier referencing
    width = combined_binary.shape[1]
    height = combined_binary.shape[0]

    # Boundaries for the ROI. Might need more tuning
    bottom_bound = height - 10
    top_bound = 450
    left_lane_end = 550
    right_lane_end = 725

    # top left, top right, bottom right, bottom left
    ROI = np.float32([[left_lane_end, top_bound], [right_lane_end, top_bound], [width, bottom_bound], [0, bottom_bound]])

    # Get the transformed image of the lane. This should be a bird's eye view of the lanes so the lanes pixels can
    # be easily identified.
    warped, Minv = getTransformedLaneView(combined_binary, ROI)
    
#     plt.imsave("output_images/warped_lanes.jpg", warped, cmap="gray")

    # Identify which pixels belong to each lane line
    lane_lines, leftx, lefty, rightx, righty, ploty, left_fitx, right_fitx = fitPolynomial(warped, left_lane, right_lane)
    
#     plt.imsave("output_images/warped_lane_lines.jpg", lane_lines)
    
    # Polulate the data for the two lines
    left_lane.detected = True
    left_lane.current_fit = left_fitx
    left_lane.allx = leftx
    left_lane.ally = lefty
#     left_lane.updateLine()
    
    right_lane.detected = True
    right_lane.current_fit = right_fitx
    right_lane.allx = rightx
    right_lane.ally = righty
#     right_lane.updateLine()
    
    # Find the curvature of each lane 
    left_rad, right_rad, offset_from_center = getLineCurvatures(width, leftx, lefty, rightx, righty, 700, 720)
    avg_rad = left_rad + right_rad / 2
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (width, height)) 
    
#     plt.imsave("output_images/unwarped_lane_lines.jpg", newwarp)
    
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    # Return the result of the pipeline
    return result, avg_rad, offset_from_center, left_lane, right_lane

In [4]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None 
        
    def updateLine(self):
        # Append latest fit to list of recents
        self.recent_xfitted.append(self.current_fit)
        
        # If there are now more than 20 fits pop off the oldest one
        if len(self.recent_xfitted) > 20:
            self.recent_xfitted.pop(0)
        
        # Take the mean of the last 20 fits to get the average best fit
        self.best_fit = np.mean(self.recent_xfitted, axis=0)
        

In [5]:
# Get the camera parameter matrix and distortion matrix
# This only needs to be run once and can take a few seconds
mtx, dist = calibrateCam("camera_cal/calibration*.jpg", 9, 6)

Calibrating from images in camera_cal/calibration*.jpg


In [6]:
# Read in the image that the pipeline will process
img = mpimg.imread("test_images/test1.jpg")

# Create two line objects to define our lane
left_lane = Line()
right_lane = Line()

# res, rad, offset, left_lane, right_lane = pipeline(img, mtx, dist, left_lane, right_lane)

# plt.imsave("output_images/final_result.jpg", res)
filename = "project_video.mp4"

# Parameters for text drawing
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
# origin 
rad_org = (img.shape[1] - 500, 50) 
offset_org = (img.shape[1] - 500, 100) 
# fontScale 
fontScale = 1
# Red in BGR
color = (0, 0, 255) 
# Line thickness of 2 px 
thickness = 2

vid = cv2.VideoCapture(filename)
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_videos/" + filename[:-4] + "_output" + filename[-4:],fourcc, 24.0, (img.shape[1], img.shape[0]))

ret = True

# Loop through the video frame by frame
while ret == True:
    ret, frame = vid.read()

    if ret == True:
        lane, rad, offset, left_lane, right_lane = pipeline(frame, mtx, dist, left_lane, right_lane)
        
        # Draw text on the frame for curvature radius and offset before writing to the video
        lane = cv2.putText(lane, "Curvature Radius: {:.02f}m".format(rad), rad_org, font, fontScale, color, thickness, cv2.LINE_AA)
        lane = cv2.putText(lane, "Offset From Center: {:.02f}m".format(offset), offset_org, font, fontScale, color, thickness, cv2.LINE_AA)

        # Write the final frame to the output video
        out.write(lane)

        
lane = cv2.cvtColor(lane, cv2.COLOR_BGR2RGB)
plt.imsave("output_images/final_result_w_text.jpg", lane)        

# Free the video stream and video writer
vid.release()
out.release()